In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
data.head()

In [ ]:
#EDA for categorical variables

fig, ax = plt.subplots(2,3, figsize=(16,9))
fig.suptitle("Categorical Variables Distribution")

#Death Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[0][0], x="DEATH_EVENT", data=data)

#Anemia Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[1][0], x="anaemia", data=data)

#Diabetes Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[0][1], x="diabetes", data=data)

#Fumadores Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[1][1], x="smoking", data=data)

#Sex Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[0][2], x="sex", data=data)

#high_blood_pressure Count
sns.set_theme(style="darkgrid")
sns.countplot(ax=ax[1][2], x="high_blood_pressure", data=data)

In [ ]:
#EDA for numerican variables

fig, ax = plt.subplots(2,3, figsize=(16,9))
fig.suptitle("Numerical Variables Distribution")

#Age Distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[0][0], x="age", data=data, kde= True, stat="count")

#Creatin distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[0][1], x="creatinine_phosphokinase", data=data, kde= True, stat="count")
                      
#serum_creatin distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[0][2],x="serum_creatinine", data=data, kde= True, stat="count")

#platelets distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[1][0],x="platelets", data=data, kde= True, stat="count")

#sodium distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[1][1],x="serum_sodium", data=data, kde= True, stat="count")

#ejection_fraction distribution
sns.set_theme(style="darkgrid")
sns.histplot(ax=ax[1][2], x = "ejection_fraction", data=data, kde= True, stat="count")

In [ ]:
data.describe()

In [ ]:
#Making grapical analisis with the categorical variables

sns.catplot(x="DEATH_EVENT", col="sex", data=data, kind="count")

sns.catplot(x="DEATH_EVENT", col="smoking", data=data, kind="count")

sns.catplot(x="DEATH_EVENT", col="high_blood_pressure", data=data, kind="count")

sns.catplot(x="DEATH_EVENT", col="diabetes", data=data, kind="count")

print("We can have an primary aproach with this catplots. It seems to be that being an smoker, having diabities and high blood pleasure increase the probability of death")

In [ ]:
#Looking for correlations in the numerical data
plt.figure(figsize=(16,9))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, cbar=True)

In [ ]:
data.isnull().sum()

In [ ]:
#Clearly the dataset is unbalanced

cant_heart_pos = len(data[data["DEATH_EVENT"]==1])
cant_heart_neg = len(data[data["DEATH_EVENT"]==0])

tasa = cant_heart_pos / len(data)

print("Only the {}% of the dataset are DEATH_EVENTS=1. We will use the SMOTE tecnique to balance the data".format(tasa*100))


In [ ]:
predictors = data.columns.values.tolist()[:12]
target = data.columns.values.tolist()[12]
X = data[predictors]
Y = data[target]

In [ ]:
#Making the prediction with Desicion Tree without features balancing

In [ ]:
data[target] = np.where(data[target]==1, "yes", "no")
Y = data[target]

In [ ]:
def tree(max_depth, min_samples_split, n_splits):
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import classification_report
    
    tree = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, 
                                  min_samples_split=min_samples_split, random_state=99)
    tree.fit(X,Y)
    
    pred = tree.predict(X)
    
    
    cv = KFold(n_splits= n_splits, shuffle=True, random_state=1)
    
    score = np.mean(cross_val_score(tree, X, Y, scoring = "accuracy", cv=cv))*100
    


    print(classification_report(Y, pred, target_names= ["NO", "YES"]))
    


In [ ]:
tree(n_splits=4, max_depth=3, min_samples_split=10)

In [ ]:
#Making the prediction with Random Forest without features balancing

In [ ]:
def rand_for(n_estimators,min_samples_leaf):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import classification_report
    
    forest_clas = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=n_estimators, 
                                         min_samples_leaf=min_samples_leaf)
    forest_clas.fit(X,Y)
    
    pred = forest_clas.predict(X)
    
    cv = KFold(n_splits= 4, shuffle=True, random_state=1)
    
    score = np.mean(cross_val_score(forest_clas, X, Y, scoring = "accuracy", cv=cv))*100
    
    print(classification_report(Y, pred, target_names= ["NO", "YES"]))
    

In [ ]:
rand_for(n_estimators=500,min_samples_leaf=20)

In [ ]:
#Making the prediction with Logistic Regression without features balancing

In [ ]:
def logistic(n_splits):
    from sklearn import linear_model
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import classification_report

    logit_model = linear_model.LogisticRegression()
    logit_model.fit(X,Y)
    
    pred = logit_model.predict(X)
    
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    scores = cross_val_score(linear_model.LogisticRegression(), X, Y, scoring="accuracy", cv=cv)
    scores


    print(classification_report(Y, pred, target_names= ["NO", "YES"]))

In [ ]:
logistic(n_splits=4)

In [ ]:
#SMOTE for unbalanced data

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)


sm = SMOTE(random_state = 2)
X_train_res, Y_train_res = sm.fit_resample(X_train, Y_train)

In [ ]:
#Decision Tree with balanced Data

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report
    
tree = DecisionTreeClassifier(criterion="entropy", max_depth=3, 
                                  min_samples_split=10, random_state=99)
tree.fit(X_train_res,Y_train_res)
pred_tree = tree.predict(X_test)

tree_smote = classification_report(Y_test, pred_tree, output_dict=True,target_names= ["NO", "YES"] )

print(classification_report(Y_test, pred_tree, target_names= ["NO", "YES"]))

In [ ]:
#Random Forest with balanced Data

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report


forest_clas = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=500, 
                                         min_samples_leaf=20)
forest_clas.fit(X_train_res,Y_train_res)
    
pred_for = forest_clas.predict(X_test)

rand_smote = classification_report(Y_test, pred_for, output_dict=True,target_names= ["NO", "YES"] )

print(classification_report(Y_test, pred_for, target_names= ["NO", "YES"]))

In [ ]:
#Logistic Regression with balanced Data

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report

logit_model = linear_model.LogisticRegression()
logit_model.fit(X_train_res,Y_train_res)
    
pred_log = logit_model.predict(X_test)

log_smote = classification_report(Y_test, pred_log, output_dict=True,target_names= ["NO", "YES"] )


print(classification_report(Y_test, pred_log, target_names= ["NO", "YES"]))

In [ ]:
recall_smote = [tree_smote["YES"]["recall"] , rand_smote["YES"]["recall"] , log_smote["YES"]["recall"]]
recall = [0.71, 0.70, 0.66] #Recall values for modeling without smote
label = ["Tree Model", "Random Forest", "Logaritm"]

accuracy_smote = [tree_smote["accuracy"] , rand_smote["accuracy"] , log_smote["accuracy"]]
accuracy = [0.86, 0.87, 0.82] #Recall values for modeling without smote

In [ ]:
plt.plot(label, recall, c="r", label="Unbalanced")
plt.plot(label, recall_smote,label="Balanced")
plt.legend(loc="upper right")
plt.title("Recall Comparision Between Unbalanced and SMOTE Balanced")
plt.show()

plt.plot(label, accuracy, c="r", label="Unbalanced")
plt.plot(label, accuracy_smote, label="Balanced")
plt.legend(loc="upper right")
plt.title("Accuracy Comparision Between Unbalanced and SMOTE Balanced")
plt.show()

In [ ]:
# Conclusion:

    #Balancing the data with SMOTE and NeverMiss tecniques doens´t have an impovement in mean accuracy in the 3 models i choose. But the random forest mantains almost with the same acc rate.
    
    #In this particulary dataset is more important to predcit as much as possible the heart attacks, even if to achive that you increase the FN rate. Balancing the data shows an highly increas on the Recall rate, that means that the model predict more heart attacks.
    
    #The best model is the Random Forest with the data balanced because it achives an an 82% of recall